In [1]:
%pip install cmem-cmempy llama-index python-dotenv llama-index-vector-stores-postgres


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv
%reload_ext dotenv

In [3]:
import os

import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
!docker pull ankane/pgvector
!docker run -d -e POSTGRES_USER=eccenca -e POSTGRES_PASSWORD=eccenca -e POSTGRES_DB=eccenca --name pgvector-eccenca -p 5432:5432 ankane/pgvector

Using default tag: latest
latest: Pulling from ankane/pgvector
Digest: sha256:956744bd14e9cbdf639c61c2a2a7c7c2c48a9c8cdd42f7de4ac034f4e96b90f8
Status: Image is up to date for ankane/pgvector:latest
docker.io/ankane/pgvector:latest

What's next:
    View a summary of image vulnerabilities and recommendations → docker scout quickview ankane/pgvector
docker: Error response from daemon: Conflict. The container name "/pgvector-eccenca" is already in use by container "560fcddb8e28282d37c2e62d80b6bd81d8866b5f2501a7705f505d207d55f3b9". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [5]:
from IPython.display import Markdown

from llama_index_cmem.readers.cmem import CMEMReader

reader = CMEMReader()
documents = reader.load_data(graph="http://ld.company.org/prod-inst/")
display(Markdown(f"### Documents loaded: __{len(documents)}__ "))

### Documents loaded: __23558__ 

In [6]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

hybrid_vector_store = PGVectorStore.from_params(
    database="eccenca",
    host="localhost",
    password="eccenca",
    port="5432",
    user="eccenca",
    table_name="prod-inst_hybrid",
    embed_dim=1536,  # openai embedding dimension
    hybrid_search=True,
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

storage_context = StorageContext.from_defaults(vector_store=hybrid_vector_store)
hybrid_index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, show_progress=True
)
hybrid_query_engine = hybrid_index.as_query_engine(vector_store_query_mode="hybrid")

/Users/rene/Documents/git/github/eccenca/llama-index-cmem/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 2048/2048 [01:20<00:00, 25.49it/s]
PG Setup: Error creating HNSW index: (psycopg2.errors.SyntaxError) syntax error at or near "-"
LINE 1: CREATE INDEX IF NOT EXISTS data_prod-inst_hybrid_embedding_i...
                                            ^

[SQL: CREATE INDEX IF NOT EXISTS data_prod-inst_hybrid_embedding_idx ON public.data_prod-inst_hybrid USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64)]
(Background on this error at: https://sqlalche.me/e/20/f405)
Generating embeddings: 100%|██████████| 1030/1030 [00:12<00:00, 83.27it/s]


In [8]:
question = "What information do you have about Liese Adam? Give me as many details as possible."
answer = hybrid_query_engine.query(question)
display(Markdown(f"## Question: _{question}_"))
display(Markdown(f"### Answer:\n\n{answer}"))

## Question: _What information do you have about Liese Adam? Give me as many details as possible._

### Answer:

Liese Adam's full name is Liese Adam and her email address is Liese.Adam@company.org.